# Analyse des données et prétraitement

In [ ]:
import os
import pandas as pd

# Chemins vers les répertoires et fichiers
data_dir = "data/audio_and_txt_files"
diagnosis_file = "data/patient_diagnosis.csv"

# Charger le fichier des diagnostics
diagnosis_df = pd.read_csv(diagnosis_file, header=None, names=["patient", "diagnosis"])

# Initialisation de la liste pour stocker les données
data = []

# Parcourir les fichiers audio et texte
for filename in os.listdir(data_dir):
    if filename.endswith(".wav"):  # Fichiers audio
        # Extraire les métadonnées à partir du nom de fichier
        parts = filename.split("_")
        patient = int(parts[0])
        recording_index = parts[1]
        chest_location = parts[2]
        acquisition_mode = parts[3]
        recording_equipment = parts[4].split(".")[0]
        
        # Associer le diagnostic
        diagnosis = diagnosis_df.loc[diagnosis_df["patient"] == patient, "diagnosis"].values[0]

        # Charger le fichier texte correspondant
        txt_filename = filename.replace(".wav", ".txt")
        txt_filepath = os.path.join(data_dir, txt_filename)
        
        if os.path.exists(txt_filepath):
            with open(txt_filepath, "r") as file:
                for line in file:
                    begin_time, end_time, crackles, wheezes = map(float, line.strip().split("\t"))
                    data.append({
                        "patient": patient,
                        "recording_index": recording_index,
                        "chest_location": chest_location,
                        "acquisition_mode": acquisition_mode,
                        "recording_equipment": recording_equipment,
                        "diagnosis": diagnosis,
                        "begin_time": begin_time,
                        "end_time": end_time,
                        "crackles": int(crackles),
                        "wheezes": int(wheezes),
                    })

# Créer un DataFrame à partir des données
df = pd.DataFrame(data)

# Sauvegarder dans un fichier CSV si nécessaire
df.to_csv("data/processed_data.csv", index=False)